In [67]:
%%capture
!pip install gdown

In [68]:
from google.colab import drive
drive.mount('/content/drive')
import torch
from torch import nn, optim
import os
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import iplot
from plotly.subplots import make_subplots
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cuda


In [69]:
BATCH_SIZE = 32 # 논문과 똑같게
LR = 0.001 # 논문과 똑같게
EPOCH = 10
LAMBDA = 0.001 # 논문과 똑같게 (l2-는 아니고 orthogonal로 만들기 위한 regularization term에 곱해짐)
criterion = nn.CrossEntropyLoss()

data_load = True # 최초 한번만 실행
new_model_train = False
hyuk_model_use = True # 여러분만의 모델 만들어서 사용하고 싶다면 False로
if hyuk_model_use:
    !gdown https://drive.google.com/uc?id=1_cKznXlYHP4R5yOcctPt3kCeFfrIOTXZ -O PointNet_ModelNet10.pt
    save_model_path = 'PointNet_ModelNet10.pt'
else:
    save_model_path = '/content/drive/MyDrive/Colab Notebooks/results/PointNet_ModelNet10_v2.pt'

Downloading...
From: https://drive.google.com/uc?id=1_cKznXlYHP4R5yOcctPt3kCeFfrIOTXZ
To: /content/PointNet_ModelNet10.pt
100% 14.0M/14.0M [00:00<00:00, 197MB/s]


## 데이터셋 만들기

In [70]:
if data_load:
    !wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
    !unzip ModelNet10.zip

class ModelNetDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.file_paths = []
        self.labels = []

        categories = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        for category in categories:
            new_dir = os.path.join(root_dir, category, split)  # Path to the train/test split directory
            for file in os.listdir(new_dir):
                if file.endswith('.off'):
                    self.file_paths.append(os.path.join(new_dir, file))
                    self.labels.append(category)

        self.label_ids = {label: idx for idx, label in enumerate(sorted(set(self.labels)))}
        self.classes = list(self.label_ids.keys())
    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx): # 데이터 하나만 쏙 빼는 함수임 따라서 여기서 point_cloud.shape = point개수,3
        file_path = self.file_paths[idx]
        point_cloud = self.load_point_cloud(file_path, n_points=1024)
        label_name = self.labels[idx]
        label_idx = self.label_ids[label_name]
        if self.transform:
            point_cloud = self.transform(point_cloud)
        point_cloud = point_cloud.transpose(0,1) # 개채포로 나오게끔
        return point_cloud, label_idx

    def load_off_file(self, file_path):
        with open(file_path, 'r') as file:
            first_line = file.readline().strip()
            # OFF 는 헤더
            n_verts, n_faces, _ = tuple([int(s) for s in file.readline().strip().split(' ')])
            # 3514 3546 0 는 vertices faces edge 수
            verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
            # 3514 번 반복하면서 verts 좌표들 읽어온다
            # faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
            # 3546 번 반복하면 face 정보 (3 1897 1895 1898 이면 vertex index 1897, 1895, 1898 로 면 하나 만드는 거) 읽을 수 있다
        return np.array(verts) # , np.array(faces) # point cloud 다룰 거니까 verts 만

    def load_point_cloud(self, file_path, n_points):
        verts = self.load_off_file(file_path)
        # Subsample or pad the point cloud to ensure a consistent size
        if len(verts) > n_points: # 너무 많으면 제한 줌 몇개만 골라
            choice = np.random.choice(len(verts), n_points, replace=False)
            verts = verts[choice, :]
        elif len(verts) < n_points: # 패딩 필요
            padding = np.zeros((n_points - len(verts), 3))
            verts = np.vstack((verts, padding))
        return torch.tensor(verts, dtype=torch.float)

def transform(point_cloud):
    point_cloud_np = point_cloud.numpy()  # Convert to NumPy array for processing

    # Normalize
    point_cloud_np -= np.mean(point_cloud_np, axis=0)
    max_distance = np.max(np.linalg.norm(point_cloud_np, axis=1))
    point_cloud_np /= max_distance

    # Random rotation around the z-axis (up-axis)
    angle = np.random.uniform(0, 2 * np.pi)
    rz = np.array([[np.cos(angle), -np.sin(angle), 0],
                   [np.sin(angle), np.cos(angle), 0],
                   [0, 0, 1]])
    point_cloud_np = np.dot(point_cloud_np, rz)

    # Jittering (adding Gaussian noise)
    jitter = np.random.normal(0, 0.02, point_cloud_np.shape)
    point_cloud_np += jitter

    return torch.tensor(point_cloud_np, dtype=torch.float)  # Convert back to tensor

In [71]:
# Create dataset and data loader
train_DS = ModelNetDataset(root_dir='/content/ModelNet10', split='train', transform=transform)
test_DS = ModelNetDataset(root_dir='/content/ModelNet10', split='test', transform=transform)
train_DL = DataLoader(train_DS, batch_size=BATCH_SIZE, shuffle=True)
test_DL = DataLoader(test_DS, batch_size=BATCH_SIZE, shuffle=True)

In [72]:
print(len(train_DS))
print(len(test_DS))

3991
908


In [73]:
x_batch, y_batch = next(iter(train_DL))
x_batch.shape

torch.Size([32, 3, 1024])

## 3d point cloud plot

In [74]:
def plot_point_cloud(data):
    trace = go.Scatter3d(
        x=data[0, :].cpu().numpy(),  # X coordinates
        y=data[1, :].cpu().numpy(),  # Y coordinates
        z=data[2, :].cpu().numpy(),  # Z coordinates
        mode='markers',
        marker=dict(
            size=2,
            color=data[2, :].cpu().numpy(),  # Color by Z value
            colorscale='Viridis',
            opacity=0.8
        )
    )
    layout = go.Layout(
        margin=dict(l=0, r=0, b=0, t=0)
    )
    fig = go.Figure(data=[trace])
    iplot(fig)

In [75]:
idx = 2
plot_point_cloud(x_batch[idx])
print(train_DS.classes[y_batch[idx]])

night_stand


## 모델 만들기

In [76]:
class Spatial_Transformer_Network(nn.Module):
    def __init__(self, k):
        super().__init__()

        self.k = k
        self.T_Net1 = nn.Sequential(nn.Conv1d(k, 64, 1),
                                    nn.BatchNorm1d(64),
                                    nn.ReLU(True),
                                    nn.Conv1d(64, 128, 1),
                                    nn.BatchNorm1d(128),
                                    nn.ReLU(True),
                                    nn.Conv1d(128, 1024, 1),
                                    nn.BatchNorm1d(1024),
                                    nn.ReLU(True))

        self.T_Net2 = nn.Sequential(nn.Conv1d(1024, 512, 1),
                                    nn.BatchNorm1d(512),
                                    nn.ReLU(True),
                                    nn.Conv1d(512, 256, 1),
                                    nn.BatchNorm1d(256),
                                    nn.ReLU(True),
                                    nn.Conv1d(256, k*k, 1))

        with torch.no_grad(): # transform matrix의 초깃값을 identity로
            self.T_Net2[-1].weight.zero_()
            identity = torch.eye(k).flatten()
            self.T_Net2[-1].bias.copy_(identity)

    def forward(self, x): # x.shape = 개채포
        transform = self.T_Net1(x)
        transform = torch.max(transform, 2, keepdim=True)[0] # 개채1
        transform = self.T_Net2(transform)
        transform = transform.squeeze(2).reshape(-1, self.k, self.k) # 개kk

        x = x.transpose(1,2) # 개채포 => 개포채
        x = x @ transform # 개포채 @ 개kk => 개포k
        x = x.transpose(1,2) # 개k포 = 개채포
        return x, transform

class PointNet(nn.Module):
    def __init__(self, num_classes = 10):
        super().__init__()
        self.input_transform = Spatial_Transformer_Network(k=3)

        self.mlp1 = nn.Sequential(nn.Conv1d(3, 64, 1),
                                  nn.BatchNorm1d(64),
                                  nn.ReLU(True),
                                  nn.Conv1d(64, 64, 1),
                                  nn.BatchNorm1d(64),
                                  nn.ReLU(True))

        self.feature_transform = Spatial_Transformer_Network(k=64)

        self.mlp2 = nn.Sequential(nn.Conv1d(64, 64, 1),
                                  nn.BatchNorm1d(64),
                                  nn.ReLU(True),
                                  nn.Conv1d(64, 128, 1),
                                  nn.BatchNorm1d(128),
                                  nn.ReLU(True),
                                  nn.Conv1d(128, 1024, 1),
                                  nn.BatchNorm1d(1024),
                                  nn.ReLU(True))

        self.classifier = nn.Sequential(nn.Conv1d(1024, 512, 1),
                                        nn.BatchNorm1d(512),
                                        nn.ReLU(True),
                                        nn.Conv1d(512, 256, 1),
                                        nn.BatchNorm1d(256),
                                        nn.ReLU(True),
                                        nn.Dropout(0.3),
                                        nn.Conv1d(256, num_classes, 1))

    def forward(self, x):
        x, A1 = self.input_transform(x)
        x = self.mlp1(x)
        x, A2 = self.feature_transform(x)
        x = self.mlp2(x)
        x = torch.max(x, 2, keepdim=True)[0] # 개채1
        x = self.classifier(x).squeeze(2)
        return x, A1, A2

class PointNetSegmentation(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.input_transform = Spatial_Transformer_Network(k=3)

        self.mlp1 = nn.Sequential(nn.Conv1d(3, 64, 1),
                                  nn.BatchNorm1d(64),
                                  nn.ReLU(True),
                                  nn.Conv1d(64, 64, 1),
                                  nn.BatchNorm1d(64),
                                  nn.ReLU(True))

        self.feature_transform = Spatial_Transformer_Network(k=64)

        self.mlp2 = nn.Sequential(nn.Conv1d(64, 64, 1),
                                  nn.BatchNorm1d(64),
                                  nn.ReLU(True),
                                  nn.Conv1d(64, 128, 1),
                                  nn.BatchNorm1d(128),
                                  nn.ReLU(True),
                                  nn.Conv1d(128, 1024, 1),
                                  nn.BatchNorm1d(1024),
                                  nn.ReLU(True))

        self.mlp3 = nn.Sequential(nn.Conv1d(1088, 512, 1),
                                  nn.BatchNorm1d(512),
                                  nn.ReLU(True),
                                  nn.Conv1d(512, 256, 1),
                                  nn.BatchNorm1d(256),
                                  nn.ReLU(True),
                                  nn.Conv1d(256, 128, 1),
                                  nn.BatchNorm1d(128),
                                  nn.ReLU(True),
                                  nn.Conv1d(128, 128, 1),
                                  nn.BatchNorm1d(128),
                                  nn.ReLU(True),
                                  nn.Conv1d(128, num_classes, 1))

    def forward(self, x):
        num_points = x.shape[2]
        x, A1 = self.input_transform(x)
        x = self.mlp1(x)
        pointwise_features, A2 = self.feature_transform(x)
        x = self.mlp2(pointwise_features)
        x = torch.max(x, 2, keepdim=True)[0] # 개채1
        global_feature = x.repeat(1, 1, num_points)
        x = torch.cat([pointwise_features, global_feature], dim=1)
        x = self.mlp3(x)
        return x, A1, A2

In [58]:
model_classifier = PointNet(num_classes=10)
print(model_classifier)
!pip install torchinfo
from torchinfo import summary
summary(model_classifier, input_size=(2,3,1024), device='cpu')

PointNet(
  (input_transform): Spatial_Transformer_Network(
    (T_Net1): Sequential(
      (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (7): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU(inplace=True)
    )
    (T_Net2): Sequential(
      (0): Conv1d(1024, 512, kernel_size=(1,), stride=(1,))
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv1d(512, 256, kernel_size=(1,), stride=(1,))
      (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5)

Layer (type:depth-idx)                   Output Shape              Param #
PointNet                                 [2, 10]                   --
├─Spatial_Transformer_Network: 1-1       [2, 3, 1024]              --
│    └─Sequential: 2-1                   [2, 1024, 1024]           --
│    │    └─Conv1d: 3-1                  [2, 64, 1024]             256
│    │    └─BatchNorm1d: 3-2             [2, 64, 1024]             128
│    │    └─ReLU: 3-3                    [2, 64, 1024]             --
│    │    └─Conv1d: 3-4                  [2, 128, 1024]            8,320
│    │    └─BatchNorm1d: 3-5             [2, 128, 1024]            256
│    │    └─ReLU: 3-6                    [2, 128, 1024]            --
│    │    └─Conv1d: 3-7                  [2, 1024, 1024]           132,096
│    │    └─BatchNorm1d: 3-8             [2, 1024, 1024]           2,048
│    │    └─ReLU: 3-9                    [2, 1024, 1024]           --
│    └─Sequential: 2-2                   [2, 9, 1]                 --
│

In [59]:
model_seg = PointNetSegmentation(num_classes=10)
# print(model_seg)
summary(model_seg, input_size=(2,3,1024), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
PointNetSegmentation                     [2, 10, 1024]             --
├─Spatial_Transformer_Network: 1-1       [2, 3, 1024]              --
│    └─Sequential: 2-1                   [2, 1024, 1024]           --
│    │    └─Conv1d: 3-1                  [2, 64, 1024]             256
│    │    └─BatchNorm1d: 3-2             [2, 64, 1024]             128
│    │    └─ReLU: 3-3                    [2, 64, 1024]             --
│    │    └─Conv1d: 3-4                  [2, 128, 1024]            8,320
│    │    └─BatchNorm1d: 3-5             [2, 128, 1024]            256
│    │    └─ReLU: 3-6                    [2, 128, 1024]            --
│    │    └─Conv1d: 3-7                  [2, 1024, 1024]           132,096
│    │    └─BatchNorm1d: 3-8             [2, 1024, 1024]           2,048
│    │    └─ReLU: 3-9                    [2, 1024, 1024]           --
│    └─Sequential: 2-2                   [2, 9, 1]                 --
│

In [60]:
model_classifier = model_classifier.to(DEVICE)
x = torch.randn(2,3,1024)
print(model_classifier(x.to(DEVICE))[0].shape)
print(model_classifier(x.to(DEVICE))[1].shape) # A1
print(model_classifier(x.to(DEVICE))[2].shape) # A2

model_seg = model_seg.to(DEVICE)
x = torch.randn(2,3,1024)
print(model_seg(x.to(DEVICE))[0].shape)
print(model_seg(x.to(DEVICE))[1].shape) # A1
print(model_seg(x.to(DEVICE))[2].shape) # A2

torch.Size([2, 10])
torch.Size([2, 3, 3])
torch.Size([2, 64, 64])
torch.Size([2, 10, 1024])
torch.Size([2, 3, 3])
torch.Size([2, 64, 64])


# 훈련 시키기

In [61]:
def orthogonal_regularizer(matrix):
    K = matrix.shape[1]
    identity = torch.eye(K).to(DEVICE)
    identity = identity.repeat(matrix.shape[0], 1, 1)
    matrix_T = matrix.transpose(1, 2)
    orthogonality_loss = torch.norm(identity - matrix @ matrix_T, dim=(1, 2))
    return orthogonality_loss.mean()

def Train(model, train_DL, criterion, optimizer, EPOCH):

    NoT=len(train_DL.dataset) # Number of training data
    loss_history = []

    model.train() # train mode로!
    for ep in range(EPOCH):
        rloss = 0
        for x_batch, y_batch in train_DL:
            x_batch = x_batch.to(DEVICE)
            y_batch = y_batch.to(DEVICE)
            # inference
            y_hat, A1, A2 = model(x_batch)
            # cross entropy loss + orthogonal matrix 정규화
            L = criterion(y_hat, y_batch)
            reg_loss = LAMBDA * (orthogonal_regularizer(A1) + orthogonal_regularizer(A2))
            loss = L + reg_loss
            # update
            optimizer.zero_grad() # gradient 누적을 막기 위한 초기화
            loss.backward() # backpropagation
            optimizer.step() # weight update
            # loss accumulation
            loss_b = loss.item() * x_batch.shape[0] # batch loss # BATCH_SIZE 로 하면 마지막 18개도 32개로 계산해버림
            rloss += loss_b # running loss
        # print loss
        loss_e = rloss/NoT # epoch loss
        loss_history += [loss_e]
        print(f"Epoch: {ep+1} train loss: {round(loss_e,3)}")
        print("-"*20)

    return loss_history

def Test(model,test_DL):
    model.eval()
    with torch.no_grad():
        rcorrect = 0
        for x_batch, y_batch in test_DL:
            x_batch = x_batch.to(DEVICE)
            y_batch = y_batch.to(DEVICE)
            # inference
            y_hat = model(x_batch)[0]
            pred = y_hat.argmax(dim=1)
            corrects_b = torch.sum(pred == y_batch).item()
            rcorrect += corrects_b
        accuracy_e = rcorrect/len(test_DL.dataset)*100
    print(f"Test accuracy: {rcorrect}/{len(test_DL.dataset)} ({round(accuracy_e,1)} %)")
    return round(accuracy_e,1)

In [62]:
if new_model_train:
    optimizer = optim.Adam(model_classifier.parameters(), lr = LR, )
    loss_history = Train(model_classifier, train_DL, criterion, optimizer, EPOCH)

    torch.save(model_classifier, save_model_path)

    plt.figure()
    plt.plot(range(1,EPOCH+1),loss_history)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Train Loss")
    plt.grid()

In [63]:
load_model = torch.load(save_model_path, map_location=DEVICE)

In [64]:
accuracy = Test(load_model, test_DL)

Test accuracy: 693/908 (76.3 %)


In [65]:
def Test_plot_point_cloud(model, test_DL):
    model.eval()
    with torch.no_grad():
        x_batch, y_batch = next(iter(test_DL))
        x_batch = x_batch.to(DEVICE)
        y_hat = model(x_batch)[0]
        pred = y_hat.argmax(dim=1)

    x_batch = x_batch.cpu()
    y_batch = y_batch.cpu()
    pred = pred.cpu()

    # Define subplot titles
    subplot_titles = []
    for idx in range(6):
        pred_class = list(test_DL.dataset.label_ids.keys())[pred[idx].item()]
        true_class = list(test_DL.dataset.label_ids.keys())[y_batch[idx].item()]
        subplot_titles.append(f'Pred: {pred_class}<br>True: {true_class}')

    # Create subplots
    fig = make_subplots(
        rows=2, cols=3,
        specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}],
               [{'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}]],
        subplot_titles=subplot_titles  # Add titles to the subplots
    )

    for idx in range(6):
        data = x_batch[idx]
        trace = go.Scatter3d(x=data[0, :].numpy(),  # X coordinates
                             y=data[1, :].numpy(),  # Y coordinates
                             z=data[2, :].numpy(),  # Z coordinates
                             mode='markers',
                             marker=dict(size=2,
                                         color=data[2, :].numpy(),  # Color by Z value
                                         colorscale='Viridis',
                                         opacity=0.8))
        row = idx // 3 + 1
        col = idx % 3 + 1
        fig.add_trace(trace, row=row, col=col)

    fig.update_layout(height=800, width=1200, showlegend=False)
    iplot(fig)

In [66]:
Test_plot_point_cloud(load_model, test_DL)